# Payoff Cliquet / Ratchet

Payoff cap/planche sur rendement (simplifie).


In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
import sys
from pathlib import Path
from IPython.display import display, Markdown

_base = Path.cwd().resolve()
for extra in (_base, _base.parent, _base.parent.parent, _base.parent.parent.parent):
    candidate = extra / "scripts"
    if candidate.exists() and str(candidate) not in sys.path:
        sys.path.insert(0, str(candidate))
        break
    candidate = extra / "notebooks" / "scripts"
    if candidate.exists() and str(candidate) not in sys.path:
        sys.path.insert(0, str(candidate))
        break
plt.style.use('seaborn-v0_8-darkgrid')

from pricing import fetch_spy_history, view_cliquet, payoff_cliquet
close_spy = fetch_spy_history()
spot_ref = float(close_spy.iloc[-1])
floor_ref, cap_ref = -0.05, 0.1


## Payoff interactif
S_T, floor/cap, sigma, r, T.


In [ ]:

sigma_slider = widgets.FloatSlider(value=0.2, min=0.01, max=1.0, step=0.01, description='Sigma')
r_slider = widgets.FloatSlider(value=0.02, min=-0.05, max=0.1, step=0.005, description='r')
T_slider = widgets.FloatSlider(value=1.0, min=0.05, max=2.0, step=0.05, description='T')

spot_slider = widgets.FloatSlider(value=spot_ref, min=spot_ref*0.6, max=spot_ref*1.4, step=1.0, description='S_T')
floor_slider = widgets.FloatSlider(value=floor_ref, min=-0.2, max=0.0, step=0.01, description='Floor')
cap_slider = widgets.FloatSlider(value=cap_ref, min=0.0, max=0.3, step=0.01, description='Cap')
output = widgets.Output()

def _update(change=None):
    with output:
        output.clear_output()
        sT, floor_val, cap_val = spot_slider.value, floor_slider.value, cap_slider.value
        view_dyn = view_cliquet(spot_ref, floor=floor_val, cap=cap_val)
        premium = float(view_dyn.get('premium', 0.0))
        payoff_val = payoff_cliquet(sT, spot_ref, floor=floor_val, cap=cap_val)
        pnl_val = payoff_val - premium
        s_grid = view_dyn['s_grid']
        payoff_grid = view_dyn['payoff']
        pnl_grid = view_dyn['pnl']
        fig, ax = plt.subplots(figsize=(7,4))
        ax.plot(s_grid, payoff_grid, label='Payoff')
        ax.plot(s_grid, pnl_grid, label='P&L net', color='darkorange')
        ax.axhline(0, color='black', linewidth=0.8)
        ax.legend(loc='best')
        ax.set_xlabel('Spot')
        ax.set_ylabel('Payoff / P&L')
        ax.set_title('Cliquet / Ratchet (simplifie)')
        plt.show()
        plt.close(fig)
        display(Markdown(f"Prime ~ {premium:.4f}

- Payoff @ S_T = {payoff_val:.4f}
- P&L net = {pnl_val:.4f}"))

for sl in (spot_slider, floor_slider, cap_slider, sigma_slider, r_slider, T_slider):
    sl.observe(_update, names='value')
_update()
display(widgets.VBox([spot_slider, floor_slider, cap_slider, sigma_slider, r_slider, T_slider, output]))
